# Clone Template Group

In this notebook, the template group will be copied within the source organization and renamed so it can be identified as the group to be shared with the target organization. A new folder will be created and all the items shared with each group will be copied, renamed, and stored in this new folder. 

## Getting Started:
1. Find the group IDs for the template groups. These can be found by navigating to the group page and looking in the URL (for example, if the URL is http://envisioning.maps.arcgis.com/home/group.html?id=a7903db4086641b98570bce5856a6364#overview, the group ID is "a7903db4086641b98570bce5856a6364". Example set-up in the project organization:
    - test-group (ID "a7903db4086641b98570bce5856a6364") contains:
        - Sample Layer
        - Sample Map
    - test-group-2 (ID "4d7ff4f81d6340428ef290b7de801204") contains:
        - Sample WebApp
2. Change the variables in the first code cell. Example variables:
    - SOURCE_URL = "https://envisioning.maps.arcgis.com"
    - SOURCE_USERNAME = "admin"
    - SOURCE_GROUP_IDS = ["a7903db4086641b98570bce5856a6364", "4d7ff4f81d6340428ef290b7de801204"]
    - COPY_PREFIX = "ACME"
3. Run the notebook cell by cell. 
    - In the 3rd code cell, you will need to enter the password for the source organization username when prompted
    - In the 5th code cell, the script will print out the existing template groups and items that will be copied over
    - In the sixth code cell, the script will print out the groups and items that were copied and renamed
4. Continuing this example, after running this script, the following should now also exist in the project organization:
    - ACME-test-group contains:
        - ACME Sample Layer
        - ACME Sample Map
    - ACME-test-group-2 contains:
        - ACME WebApp
    - ACME Content, which is a folder in content, contains:
        - ACME Sample Layer
        - ACME Sample Map
        - ACME WebApp

In [ ]:
# Global Variables set by user **change these values before running**

# URL of the source portal with the template content
SOURCE_URL = "https://my-enterprise.maps.arcgis.com"

# username for source portal log-on (admin)
SOURCE_USERNAME = "my_user_name"

# source template content groups that will be shared with delivery org #aec-test-group, aec-test-group-2
SOURCE_GROUP_IDS = ["6fd586038d76406594d72ed2cb348e41"]

# the prefix that will be appended to the front to distinguish these copies of the template items and groups
COPY_PREFIX = "my_prefix_"

In [ ]:
# import libraries
from arcgis.gis import GIS
from IPython.display import display
from arcgis.gis import Group
from arcgis.gis import Item
import tempfile

In [ ]:
# connection to Project organization and setting naming conventions
print("Connecting to {} with username {}...".format(SOURCE_URL, SOURCE_USERNAME))
project = GIS(SOURCE_URL, SOURCE_USERNAME, verify_cert=False)

FOLDER_SUFFIX = "Content"
GROUP_NAME_STRUCTURE = COPY_PREFIX + "-{}"
NAME_STRUCTURE = COPY_PREFIX + " {}"
FOLDER = NAME_STRUCTURE.format(FOLDER_SUFFIX)

In [ ]:
# Creation of functions to be used later
def search_by_title(target, title, obj_type):
    """search org for an existing item with title
    args:
    target -- target GIS to search
    title -- item or group to search
    obj_type -- can be "item" or "group"
    """
    try:
        if(obj_type == "group"):
            s_items = target.groups.search(title)
        elif(obj_type == "item"):
            s_items = target.content.search(query='title:{}'.format(title))
        for s_item in s_items:
            if s_item.title == title:
                return s_item
        return None
    except Exception as e:
        print("Search by title failed with args {}, {} and {}: {}".format(target, title, obj_type, e))

def modify_item(item, target):
    """Rename cloned items"""
    try:
        title = NAME_STRUCTURE.format(item.title)
        while search_by_title(target, title, "item"):
            title = input("Title `{0}` for ITEM `{1}` already exists \nNew title: ".format(title, item.title))
        return {"title": title}
    except Exception as e:
        print("Modify Item failed with args {} and {}: {}".format(item, target, e))

def modify_group(expected_title, target):
    """Rename cloned groups"""
    try:
        title = GROUP_NAME_STRUCTURE.format(expected_title)
        while search_by_title(target, title, "group"):
            title = input("Title `{0}` for GROUP `{1}` already exists \nNew title: ".format(title, expected_title))
        return title
    except Exception as e:
        print("Modify Group failed with args {} and {}: {}".format(expected_title, target, e))

def clone_group_modify(group, target):
    """Clone group and call renaming function"""
    try:
        new_group_title = modify_group(group.title, target)
        with tempfile.TemporaryDirectory() as temp_dir:
            thumbnail_file = group.download_thumbnail(temp_dir)
            target_group = target.groups.create(title=new_group_title, tags=group.tags,
                                                description=group.description, snippet=group.snippet,
                                                access=group.access, thumbnail=thumbnail_file,
                                                is_invitation_only=True, sort_field='avgRating',
                                                sort_order='asc', is_view_only=False)
            display(target_group)
        return target_group
    except Exception as e:
        print('Group {} could not be created: {}'.format(new_group_title, e))
        return None
        
def clone_items_modify(group, target, **kwargs):
    """Clone groups and items to a target GIS
    * Abstraction over arcgis.gis.ContentManager.clone_items that also renames items and groups

    args:
    group -- group containing items to be cloned. cloned group name based off 'GROUP_NAME_STRUCTURE'
    target -- target gis where items or groups will be cloned
    **kwargs:
    all additional args go into the gis.ContentManager.clone_items function described here:
    https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.gis.toc.html?highlight=clone_items#arcgis.gis.ContentManager.clone_items
    """
    try:
        # clone the group
        new_group = clone_group_modify(group, target)
        group_map = {group.id: new_group.id} # put cloned items in the new group just created

        # clone items in the specified group to the folder
        items = [group]
        print("Cloning items...")
        results = target.content.clone_items(items, group_mapping=group_map, **kwargs)

        # update each result
        for result in results:
            if isinstance(result, Item):
                props = modify_item(result, target)
                args = {}
                args['data'] = props.pop('data', None)
                args['thumbnail'] = props.pop('thumbnail', None)
                args['metadata'] = props.pop('metadata', None)
                args['item_properties'] = props
                result.update(**args)
            else:
                print(result)
        return results
    except Exception as e:
        print("Clone Items modify failed with arguments {}, and {}: {}".format(group, target, e))

In [ ]:
# get and print template project groups to copy in the project org
project_groups = [Group(project, ids) for ids in SOURCE_GROUP_IDS]

for group in project_groups:
    display(group)
    print(group.content())

In [ ]:
# copy and rename project groups in project org (create external-facing version)
for group in project_groups:
    clone_res = clone_items_modify(group, project, copy_data=False, search_existing_items=False, folder=FOLDER)
    display(clone_res)